In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

In [48]:
directors = []
mpaaRatings = []
X = [] # This holds the movie data that the model will use to train.
y = [] # This holds whether the movies won an oscar or not that the model will use to train.

dummyData = pd.read_csv("oscarsDummyData.csv", encoding = "ISO-8859-1")
n = len(dummyData.index)

for i in range(0, n):
    movie = list(dummyData.iloc[i])
    director = movie[7]
    mpaaRating = movie[10]
    
    if not(director in directors):
        directors.append(director)
    movie[7] = directors.index(director) # replace director to its unique numerical value
    
    if not(mpaaRating in mpaaRatings):
        mpaaRatings.append(mpaaRating)
    movie[10] = mpaaRatings.index(mpaaRating) # replace mpaa rating to its unique numerical value
    
    X.append(movie[3:-1])
    
    if movie[1]:
        y.append(1)
    else:
        y.append(0)

# train_test_split takes the datasets, shuffles the datasets, and splits it into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True, stratify = y)


In [49]:
classifier = RandomForestClassifier(n_estimators= 100, criterion = "entropy", random_state =0)
classifier.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [50]:
predictions = classifier.predict(X_test)
pretty_list = "\n".join("{}\t\t{}".format(x, y) for x, y in zip(predictions, y_test))
print("Predictions\tActual")
print(pretty_list)

Predictions	Actual
0		0
0		0
0		0
0		0
0		0
0		0
0		0
1		1
0		0
0		0
0		0
0		0
0		0
1		1
0		0
0		0
0		0
1		1
0		0
0		1
0		0
0		0
0		0
0		0
0		0


In [51]:

# This predict_proba function produces the probability of the movies winning an oscar.
predictions_percent = classifier.predict_proba(X_test)[:,1]
pretty_list = "\n".join("{:.2f}\t{}".format(x, y) for x, y in zip(predictions_percent, y_test))
print("% of 1 \tActual")
print(pretty_list)

% of 1 	Actual
0.02	0
0.02	0
0.09	0
0.00	0
0.00	0
0.01	0
0.03	0
0.84	1
0.01	0
0.01	0
0.02	0
0.02	0
0.00	0
0.80	1
0.00	0
0.00	0
0.02	0
0.84	1
0.02	0
0.35	1
0.03	0
0.01	0
0.01	0
0.00	0
0.00	0


In [52]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm=confusion_matrix(y_test, predictions)
print(cm)
accuracy_score(y_test,predictions)

[[21  0]
 [ 1  3]]


0.96

# New Section